# Data Collection

## Web Scraping URLs

In [1]:
%pip install requests beautifulsoup4 fake_useragent pandas openpyxl

  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
  Using cached fake_useragent-1.5.1-py3-none-any.whl (17 kB)
  Using cached pandas-2.2.2-cp311-cp311-macosx_11_0_arm64.whl (11.3 MB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
  Using cached charset_normalizer-3.3.2-cp311-cp311-macosx_11_0_arm64.whl (118 kB)
  Using cached idna-3.7-py3-none-any.whl (66 kB)
  Using cached urllib3-2.2.2-py3-none-any.whl (121 kB)
  Using cached certifi-2024.7.4-py3-none-any.whl (162 kB)
  Using cached soupsieve-2.5-py3-none-any.whl (36 kB)
  Using cached numpy-2.0.1-cp311-cp311-macosx_11_0_arm64.whl (13.3 MB)
  Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)

[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the

In [2]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import urllib.parse

def google_search(query, num_results, time_filter = None):
    ua = UserAgent()
    headers = {'User-Agent': ua.random}

    query = urllib.parse.quote_plus(query)

    google_url = f"https://www.google.com/search?q={query}&num={num_results}"

    if time_filter:
        google_url += f"&tbs={time_filter}"

    response = requests.get(google_url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        search_results = []

        for g in soup.find_all('div', class_='g'):
            anchors = g.find_all('a')
            if anchors:
                link = anchors[0]['href']
                search_results.append(link)
                
        return search_results
    else:
        print(f"failed to retrieve search results: status code {response.status_code}")
        return []


In [14]:
def generate_quarters(start_year, end_year):
    quarters = {}
    if end_year == 2024:
        quarters["2024 Q1"] = "cdr:1,cd_min:1/1/2024,cd_max:3/31/2024"
        quarters["2024 Q2"] = "cdr:1,cd_min:4/1/2024,cd_max:6/30/2024"
        end_year -= 1
    for year in range(start_year, end_year + 1):
        quarters[f"{year} Q1"] = f"cdr:1,cd_min:1/1/{year},cd_max:3/31/{year}"
        quarters[f"{year} Q2"] = f"cdr:1,cd_min:4/1/{year},cd_max:6/30/{year}"
        quarters[f"{year} Q3"] = f"cdr:1,cd_min:7/1/{year},cd_max:9/30/{year}"
        quarters[f"{year} Q4"] = f"cdr:1,cd_min:10/1/{year},cd_max:12/31/{year}"
    return quarters

def generate_query(source_list):
    dictionary = {}
    for source in source_list:
        if source in dictionary:
            continue
        else:
            dictionary[source] = f"singapore industrial property market {source}"
    return dictionary

In [17]:
import pandas as pd

query_dictionary = generate_query(["cna", "singapore business review"])

quarter_dictionary = generate_quarters(2020, 2024)

headers = ["URLs", "Source", "Quarter"]
df = pd.DataFrame(columns=headers)

for source, query in query_dictionary.items():
    for quarter, time_filter in quarter_dictionary.items():
        results = google_search(query, num_results=20, time_filter=time_filter)
        temp_df = pd.DataFrame({"URLs": results, "Source": source, "Quarter": quarter})
        df = pd.concat([df, temp_df], ignore_index=True)

print(df)

query_dictionary = generate_query(["straits times", "business times", "edgeprop"])

quarter_dictionary = generate_quarters(2020, 2024)

for source, query in query_dictionary.items():
    for quarter, time_filter in quarter_dictionary.items():
        results = google_search(query, num_results=30, time_filter=time_filter)
        temp_df = pd.DataFrame({"URLs": results, "Source": source, "Quarter": quarter})
        df = pd.concat([df, temp_df], ignore_index=True)

print(df)

df.to_excel("urls.xlsx", index=False)

                                                  URLs  \
0    https://www.channelnewsasia.com/singapore/mark...   
1          https://www.youtube.com/watch?v=JsMbvmY9FWA   
2    https://www.channelnewsasia.com/singapore/joho...   
3          https://www.youtube.com/watch?v=iJ2Xfc0xnZk   
4    https://m.facebook.com/ChannelNewsAsia/posts/8...   
..                                                 ...   
432  https://plbinsights.com/the-rise-of-self-stora...   
433  https://www.propertyguru.com.sg/property-guide...   
434  https://sbr.com.sg/commercial-property/in-focu...   
435  https://realestateasia.com/videos/proptech-kuc...   
436  https://cosysingapore.com/commercial-property-...   

                        Source  Quarter  
0                          cna  2024 Q1  
1                          cna  2024 Q1  
2                          cna  2024 Q1  
3                          cna  2024 Q1  
4                          cna  2024 Q1  
..                         ...      ...  
432  sing

## URLs Cleaning

In [20]:
df = pd.read_excel("urls.xlsx")
duplicates = df["URLs"].duplicated(keep=False)
df = df[~duplicates]

with pd.ExcelWriter("urls.xlsx", engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    df.to_excel(writer, sheet_name="2020 - 2024", index=False)